In [1]:
#!/usr/local/bin/env python
from __future__ import division
import time
from copy import deepcopy
import numpy as np
from numpy.random import random_sample as rs
from scipy.stats.mstats import mquantiles as mq
from radd import build, fit, vis, models

In [67]:
build=reload(build)
models=reload(models)
fit=reload(fit)
vis=reload(vis)
m = build.Model(data=prodf_store.prodata, inits=inits, kind='xpro', fit_on='average', depends_on={'v': 'pGo'})
m.make_simulator()
ss = m.sim


      Model is prepared to fit on average xpro data, allowing
      Drift-Rate to vary across levels of pGo (0, 20, 40, 60, 80, 100) 

      I wish you a steep gradient, and a happy Nature paper 


In [74]:
m.inits['xb']=np.zeros_like(m.inits['xb'])

In [75]:
dvg = ss.simulate_pro(ss.inits, analyze=False)
rt = ss.new_analyze_pro(dvg, ss.inits)
#yhat = ss.new_analyze_pro(dvg, ss.inits)

In [70]:
rt = ss.analyze_pro(dvg, ss.inits)

In [71]:
rt

array([ 0.01920768,  0.01560624,  0.01680672,  0.01140456,  0.01380552,
        0.01380552,  0.4548707 ,  0.4728707 ,  0.4868707 ,  0.5013707 ,
        0.5233707 ,  0.4553707 ,  0.4728707 ,  0.4871207 ,  0.5003707 ,
        0.5233707 ])

## Prepare Data/Params

In [25]:
data = redf_store['redata']
binits = { k: v for k, v in redf_store['rebsl_boot_popt'].items() }
binits['ssv'] = -abs(binits['ssv'])
pinits = { k: v for k, v in redf_store['repnl_boot_popt'].items() }

avg_inits = { k: np.mean([binits[k], pinits[k]]) for k in binits.keys() }
if 'ssd' in avg_inits.keys(): del avg_inits['ssd']

##Reactive

In [4]:
m = build.Model(data=prodf_store.prodata, inits=avg_inits, kind='xpro', fit_on='average', depends_on={'v': 'pGo'})


      Model is prepared to fit on average xpro data, allowing
      Drift-Rate to vary across levels of pGo (0, 20, 40, 60, 80, 100) 

      See, it's not that simplex... 


In [ ]:
m.opti

In [51]:
dirpath = pth+'ELife/TESTX/'
if not os.path.isdir(dirpath):
      os.mkdir(dirpath)
os.chdir(dirpath)

data = redf_store['redata']
inits = dict(deepcopy(avg_inits))
inits['xb']=.01

In [14]:
inits['ssv']=-1

In [56]:
m = build.Model(data=data, inits=inits, kind='xradd', fit_on='average', depends_on={'v': 'Cond'}, dynamic='exp')


      Model is prepared to fit on average xradd data, allowing
      Drift-Rate to vary across levels of Cond (bsl, pnl) 

      I wish you a steep gradient, and a happy Nature paper 


In [57]:
m.optimize(ntrials=10000, ftol=1.e-4, xtol=1.e-4, maxfev=5000)

Optimization terminated successfully.
         Current function value: 0.001106
         Iterations: 46
         Function evaluations: 125
Optimization terminated successfully.
         Current function value: 0.003341
         Iterations: 20
         Function evaluations: 47


In [37]:
m.sim.xtb

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [47]:
m.inits

{'a': 0.44409409206000017,
 'ssv': -0.94733930097466645,
 'tr': 0.30487069814466666,
 'v': 1.0917075579863329,
 'xb': 0.01,
 'z': 0.15424655482600008}

In [49]:
m.make_simulator()
m.inits
m.sim.simulate_radd(m.inits)

array([ 0.9535 ,  0.999  ,  0.9916 ,  0.9572 ,  0.5322 ,  0.1308 ,
        0.51337,  0.54137,  0.56337,  0.58637,  0.61845,  0.50537,
        0.53137,  0.55037,  0.57137,  0.60137,  0.9538 ,  0.9984 ,
        0.9918 ,  0.96   ,  0.5484 ,  0.1354 ,  0.51437,  0.54337,
        0.56437,  0.58737,  0.61937,  0.50637,  0.53137,  0.55237,
        0.57237,  0.60537])

In [58]:
m.fitinfo

{'AIC': -284.86895215105977,
 'BIC': -281.9374803454603,
 'CNVRG': True,
 'a': 0.44481771491408967,
 'chi': 0.003603111961929814,
 'nfev': 47,
 'rchi': 0.00012010373206432714,
 'ssv': -0.85791441856599793,
 'tr': 0.30600738270484251,
 'v_bsl': 1.1041743503039452,
 'v_pnl': 1.0652308359839426,
 'xb': 0.012414171566161925,
 'z': 0.15083624322328537}

In [59]:
fits = m.fits.reshape(2,16)
#yhat = m.simulate().reshape(2, 16)
for i in range(m.ncond):
      vis.plot_fits(m.avg_y[i], fits[i],save=False)